# Step 2: Voltage Glitching to Bypass Password

**SUMMARY:** *We've seen how voltage glitching can be used to corrupt calculations, just like clock glitching. Let's continue on and see if it can also be used to break past a password check. This step will be mostly self-guided. I will be relying on you to implement and evaluate the attack!*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching

## Firmware

Again, we've already covered this in the previous step of the lab, so it'll be mostly up to you!

In [1]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
SS_VER = 'SS_VER_2_1'

In [ ]:
%run "Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ./../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
fw_path = "./../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=="SS_VER_2_1":
    target.reset_comms()

In [ ]:
scope.io.clkout = 7.5E6
def reboot_flush():            
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = "high_z"
    time.sleep(0.05)
    #Flush garbage too
    target.flush()
    

In [ ]:
#Do glitch loop
reboot_flush()
pw = bytearray([0x74, 0x6F, 0x75, 0x63, 0x68])
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)

Like with clock glitching, the scope object can set some typical glitch settings for you:

In [ ]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

In [ ]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})

Use your findings from the prior labs to replace the `???' to launch the attack. See if you can bypass the password loop from Lab 1 to log into the board without the password. Hint: Use the attack from the prior step as a reference for how to fill everything in here.

In [ ]:
import re
import struct
gc.set_range("ext_offset", 0, 150)
g_step = 0.4
gc.set_global_step(g_step)

#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

gc.set_range("width", ???, ???)
gc.set_range("offset", ???, ???)
gc.set_global_step(???)

gc.set_step("ext_offset", 1) # check each clock cycle

scope.adc.timeout = 0.1

reboot_flush()

for glitch_settings in gc.glitch_values():
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]
    scope.glitch.ext_offset = glitch_settings[2]
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        #print("Trigger still high!")
        gc.add("reset")
        reboot_flush()

    scope.arm()
    target.simpleserial_write('p', bytearray([0]*5))
    ret = scope.capture()
    scope.io.vglitch_reset()
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    else:
        val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10, timeout=50)#For loop check
        if ???: # When do we know the glitch failed and reset the device to try again?
            gc.add("reset")
        else:
            if ???: #for loop check -- How can we check if we succeeded in our attack to fall through the for loop?
                successes +=1 
                gc.add("success")
                print(val)
                print(val['payload'])
                print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
                print("🐙", end="")
            else:
                gc.add("normal")
                    
#reenable logging
cw.set_all_log_levels(cw.logging.WARNING)

Let's see where we needed to target for our glitch to work:

In [ ]:
gc.calc(["width", "offset"], "success_rate")

In [ ]:
scope.dis()
target.dis()

Congratulations on finishing the lab!

**Get sign-off from the TA for this portion of the lab.**